In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

We are going to use BTC/USDT data from the Feb 06, 2024 data in this notebook. Each record shows a transaction in which a person wants to sell with `best_ask_price` and get a deal with `best_bid_price`.
The same goes for the quantity, seller wants to sell `best_ask_qty` but buyer only buys `best_bid_qty`.

In [2]:
df = pd.read_csv('BTCUSDT-bookTicker-2024-02-06.csv')

In [3]:
df.head()

,update_id,best_bid_price,best_bid_qty,best_ask_price,best_ask_qty,transaction_time,event_time
0,3936653543857,42692.9,0.587,42693.0,5.645,1707177600000,1707177600004
1,3939719410110,43233.4,0.846,43233.6,10.399,1707235233178,1707235233182
2,3936653543872,42692.9,0.584,42693.0,5.645,1707177600000,1707177600004
3,3939719410130,43233.4,0.846,43233.6,10.636,1707235233178,1707235233182
4,3936653543878,42692.9,0.562,42693.0,5.645,1707177600000,1707177600004


In [4]:
df['transaction_time'] = pd.to_datetime(df['transaction_time'], unit='ms')

In [5]:
df.head()

,update_id,best_bid_price,best_bid_qty,best_ask_price,best_ask_qty,transaction_time,event_time
0,3936653543857,42692.9,0.587,42693.0,5.645,2024-02-06 00:00:00.000,1707177600004
1,3939719410110,43233.4,0.846,43233.6,10.399,2024-02-06 16:00:33.178,1707235233182
2,3936653543872,42692.9,0.584,42693.0,5.645,2024-02-06 00:00:00.000,1707177600004
3,3939719410130,43233.4,0.846,43233.6,10.636,2024-02-06 16:00:33.178,1707235233182
4,3936653543878,42692.9,0.562,42693.0,5.645,2024-02-06 00:00:00.000,1707177600004


In [6]:
df['event_time'] = pd.to_datetime(df['event_time'], unit='ms')

In [7]:
df.head()

,update_id,best_bid_price,best_bid_qty,best_ask_price,best_ask_qty,transaction_time,event_time
0,3936653543857,42692.9,0.587,42693.0,5.645,2024-02-06 00:00:00.000,2024-02-06 00:00:00.004
1,3939719410110,43233.4,0.846,43233.6,10.399,2024-02-06 16:00:33.178,2024-02-06 16:00:33.182
2,3936653543872,42692.9,0.584,42693.0,5.645,2024-02-06 00:00:00.000,2024-02-06 00:00:00.004
3,3939719410130,43233.4,0.846,43233.6,10.636,2024-02-06 16:00:33.178,2024-02-06 16:00:33.182
4,3936653543878,42692.9,0.562,42693.0,5.645,2024-02-06 00:00:00.000,2024-02-06 00:00:00.004


Group by hour to see the number of transactions in each hour

In [8]:
grouped = df.groupby([df['transaction_time'].dt.hour])

In [9]:
grouped['update_id'].count()

transaction_time
0      883747
1      955074
2     1165062
3      867975
4      849887
5      814240
6      829777
7      809152
8     1451969
9     1123615
10    1620430
11    1307003
12    1283219
13    1412374
14    1482887
15    1855624
16    1921017
17    1283150
18    1077940
19    1015262
20    1160909
21     894236
22     663772
23     811752
Name: update_id, dtype: int64

In [10]:
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

Find the best `k` for clustering based on the distribution score (which is based on coherence).
To have the required accuracy, we normalize the columns that we are going to use in KMeans.

In [11]:
df_bid_ask = df[['best_bid_price', 'best_bid_qty', 'best_ask_price', 'best_ask_qty']]
df_bid_ask=(df_bid_ask-df_bid_ask.mean())/df_bid_ask.std()

In [12]:
df_bid_ask

,best_bid_price,best_bid_qty,best_ask_price,best_ask_qty
0,-1.397036,-0.947282,-1.397062,-0.123554
1,1.415551,-0.903175,1.416043,0.621299
2,-1.397036,-0.947793,-1.397062,-0.123554
3,1.415551,-0.903175,1.416043,0.658433
4,-1.397036,-0.951539,-1.397062,-0.123554
...,...,...,...,...
27540068,1.416072,-1.035666,1.416043,0.171003
27540069,1.416072,-1.035666,1.416043,0.359018
27540070,1.416072,-1.035666,1.416043,0.463680
27540071,1.416072,-1.035666,1.416043,0.519301


In [ ]:
kmeans = KMeans(random_state=42, n_init="auto")
visualizer = KElbowVisualizer(kmeans, k=(2, 10))
visualizer.fit(df_bid_ask)
visualizer.show()

In [ ]:
kmeans = KMeans(n_clusters=4).fit(df_bid_ask)

In [ ]:
kmeans.cluster_centers_

Provide total best bid by multiplying price into quantity.

In [ ]:
df['best_bid_total'] = df['best_bid_price'] * df['best_bid_qty']

We are going to see the distribution of the total best bid over time.

In [ ]:
df_group_by_timestamp = df.groupby('transaction_time').mean().reset_index()
plt.plot(df_group_by_timestamp['transaction_time'], df_group_by_timestamp['best_bid_total'])

The mean distribution contains peaks which we need to investigate more and see why they are happing?

In [ ]:
df_group_by_timestamp = df.groupby('transaction_time').count().reset_index()
plt.plot(df_group_by_timestamp['transaction_time'], df_group_by_timestamp['best_bid_total'])

The count distribution, unlike the mean, doesn't contain any peak which shows these peak shouldn't happen because of low number of transactions.

In [ ]:
df_group_by_timestamp = df.groupby('transaction_time').median().reset_index()
plt.plot(df_group_by_timestamp['transaction_time'], df_group_by_timestamp['best_bid_total'])

Create another Data frame for the outliers based on their total bid.

Based on the above diagrams, it seems we have some transaction which is not normal in our context. I am going to use histogram and see the distribution of the transactions value.

In [ ]:
counts, bins = np.histogram(df['best_bid_total'])

plt.hist(bins[:-1], bins, weights=counts)

In [ ]:
bins, counts

Based on the histogram, it seems we have some actual outlier which we need to investigate. Let use KMeans and see its clustering with these outliers. For visualization of the clustering we can also used **LDA**.

In [ ]:
labels = kmeans.predict(df_bid_ask)

In [ ]:
centroids = kmeans.cluster_centers_
u_labels = np.unique(labels)
 
for i in u_labels:
    plt.scatter(df[labels == i]['best_bid_price'] , df[labels == i]['best_bid_qty'] , label = i)

It seems data is clustered using price, which is predictable because the only feature we have is price.

In [ ]:
centroids = kmeans.cluster_centers_
u_labels = np.unique(labels)
 
for i in u_labels:
    plt.scatter(df[labels == i]['best_bid_price'] , df[labels == i]['best_ask_price'] , label = i)

Based on these plots, seems data is only clustered by the `best_bid_price` which justifies the peaks we have in our day.

Based on the clustering we figured out that `best_bid_price` and `best_ask_price` are always similar, so we went for another metrics `best_spread_price` and `best_spread_qty` which is the difference between `bid` and `ask` prices and show the market status. We have different type of order books, we need to know which one here we have.

In [ ]:
df['best_spread_price'] = df['best_ask_price'] - df['best_bid_price']

df_group_by_timestamp = df.groupby('transaction_time').median().reset_index()

plt.plot(df_group_by_timestamp['transaction_time'], df_group_by_timestamp['best_spread_price'])

In [ ]:
df[df['best_spread_price'] > 30]

In [ ]:
df['best_spread_qty'] = df['best_ask_qty'] - df['best_bid_qty']

df_group_by_timestamp = df.groupby('transaction_time').median().reset_index()

plt.plot(df_group_by_timestamp['transaction_time'], df_group_by_timestamp['best_spread_qty'])

There are records in which seller wants to sell less than 1 coin but buyer, buys around 100 coins! which is insance.

In [ ]:
df[df['best_spread_qty'] < -100]

In [ ]:
df_outliers = df_group_by_timestamp[df_group_by_timestamp['best_bid_total'] > 2e6]
df_outliers

It seems, people may buy more or less than from the specified quantity (based on `ask`).

In [ ]:
centroids = kmeans.cluster_centers_
u_labels = np.unique(labels)
 
for i in u_labels:
    plt.scatter(df[labels == i]['best_spread_price'] , df[labels == i]['best_spread_qty'] , label = i)

The above diagram shows that our current clustering is not good enough. I am going to train it again using our new `spread` columns. Besides that it shows people change the amount in the lower prices.

In [ ]:
kmeans = KMeans(n_clusters=6).fit(df[['best_spread_price', 'best_spread_qty']])
labels = kmeans.predict(df[['best_spread_price', 'best_spread_qty']])

In [ ]:
centroids = kmeans.cluster_centers_
u_labels = np.unique(labels)

for i in u_labels:
    plt.scatter(df[labels == i]['best_spread_price'] , df[labels == i]['best_spread_qty'] , label = i)
plt.scatter(centroids[:,0] , centroids[:,1] , s = 80, color = 'k')
plt.legend()

In [ ]:
np.histogram(kmeans.predict(df_outliers[['best_spread_price', 'best_spread_qty']]))

All the outliers lives in the label '3' which makes sense, because all of them buying more than what seller wants to sale.

The new clustering makes more sense because it divides people based on the amount they buy in a transaction, not the price.

In [ ]:
df_ada = pd.read_csv('ADAUSDT-bookTicker-2024-02-06.csv')
df_ada['transaction_time'] = pd.to_datetime(df_ada['transaction_time'], unit='ms')
df_ada['event_time'] = pd.to_datetime(df_ada['event_time'], unit='ms')

In [ ]:
df_ada['best_spread_price'] = df_ada['best_ask_price'] - df_ada['best_bid_price']
df_ada['best_spread_qty'] = df_ada['best_ask_qty'] - df_ada['best_bid_qty']

In [ ]:
df_ada.head()

In [ ]:
df_ada_group_by_timestamp = df_ada.groupby('transaction_time').median().reset_index()
plt.plot(df_ada_group_by_timestamp['transaction_time'], df_ada_group_by_timestamp['best_spread_price'])

In [ ]:
plt.plot(df_ada_group_by_timestamp['transaction_time'], df_ada_group_by_timestamp['best_spread_qty'])

In [ ]:
ada_and_btc = pd.DataFrame()
ada_and_btc['btc'] = (df.sort_values('transaction_time'))['best_bid_price']
ada_and_btc['ada'] = (df_ada.sort_values('transaction_time'))['best_bid_price']

In [ ]:
ada_and_btc.corr()

In [ ]:
plt.plot(ada_and_btc['btc'], ada_and_btc['ada'])

In [ ]:
ada_and_btc['ada']

In [ ]:
ada_and_btc['btc']

In [ ]:
df['total_bid_price'] = df['best_bid_price'] * df['best_bid_qty']
df_ada['total_bid_price'] = df_ada['best_bid_price'] * df_ada['best_bid_qty']

df['best_bid_price_normal'] = (df['best_bid_price'] - df['best_bid_price'].min()) / (df['best_bid_price'].max() - df['best_bid_price'].min())
df_ada['best_bid_price_normal'] = (df_ada['best_bid_price'] - df_ada['best_bid_price'].min()) / (df_ada['best_bid_price'].max() - df_ada['best_bid_price'].min())

df_group_by_timestamp = df.groupby('transaction_time').median().reset_index()
df_ada_group_by_timestamp = df_ada.groupby('transaction_time').median().reset_index()

plt.plot(df_ada_group_by_timestamp['transaction_time'], df_ada_group_by_timestamp['best_bid_price_normal'], label='ada')
plt.plot(df_group_by_timestamp['transaction_time'], df_group_by_timestamp['best_bid_price_normal'], label='btc')
plt.legend()
plt.show()

In [ ]:
ada_and_btc = pd.DataFrame()
ada_and_btc['btc'] = (df_group_by_timestamp.set_index('transaction_time'))['best_bid_price']
ada_and_btc['ada'] = (df_ada_group_by_timestamp.set_index('transaction_time'))['best_bid_price']
ada_and_btc.dropna().corr()

In [ ]:
df[df['best_bid_price'] == 43000]

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, acf

In [ ]:
df_group_by_minute = df.groupby([df['transaction_time'].dt.minute]).mean()

result_adf = adfuller(df_group_by_minute.set_index('transaction_time')['best_bid_price'])
print("ADF Statistic:", result_adf[0])
print("p-value:", result_adf[1])